# E-Commerce Dataset Cleaning

This notebook handles the cleaning and preprocessing of the e-commerce dataset for the product recommendation system.

## Objectives:
- Load and explore the dataset
- Remove duplicates and handle missing values
- Standardize data formats
- Prepare data for vectorization
- Export cleaned data to database

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import sys
import re
from pathlib import Path

# Add parent directory to path to import services
sys.path.append('..')
from services.database import DatabaseService

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Load dataset from zip file
data_path = '../data/dataset.zip'

if os.path.exists(data_path):
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        # List files in zip
        file_list = zip_ref.namelist()
        print("Files in dataset.zip:")
        for file in file_list:
            print(f"  - {file}")
        
        # Extract to temporary directory
        zip_ref.extractall('../data/temp')
        
        # Find CSV files
        csv_files = [f for f in file_list if f.endswith('.csv')]
        if csv_files:
            # Load the first CSV file
            csv_file = csv_files[0]
            df = pd.read_csv(f'../data/temp/{csv_file}')
            print(f"\nLoaded dataset: {csv_file}")
            print(f"Shape: {df.shape}")
        else:
            print("No CSV files found in the zip archive")
            # Create sample data for demonstration
            df = create_sample_data()
else:
    print("Dataset.zip not found. Creating sample data for demonstration.")
    df = create_sample_data()

def create_sample_data():
    """Create sample e-commerce data for demonstration"""
    np.random.seed(42)
    
    # Sample product categories and descriptions
    categories = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books', 'Beauty']
    electronics = ['Wireless Headphones', 'Smartphone', 'Laptop Computer', 'Tablet', 'Smart Watch', 'Bluetooth Speaker']
    clothing = ['T-Shirt', 'Jeans', 'Dress', 'Jacket', 'Sneakers', 'Hat']
    home_garden = ['Coffee Maker', 'Vacuum Cleaner', 'Plant Pot', 'Lamp', 'Cushion', 'Candle']
    sports = ['Yoga Mat', 'Dumbbells', 'Running Shoes', 'Water Bottle', 'Fitness Tracker', 'Tennis Racket']
    books = ['Programming Book', 'Novel', 'Cookbook', 'Biography', 'Science Book', 'Art Book']
    beauty = ['Face Cream', 'Lipstick', 'Shampoo', 'Perfume', 'Nail Polish', 'Moisturizer']
    
    all_products = electronics + clothing + home_garden + sports + books + beauty
    countries = ['USA', 'UK', 'Canada', 'Australia', 'Germany', 'France', 'Japan', 'China']
    
    # Generate sample data
    n_samples = 1000
    data = {
        'StockCode': [f'SKU{i:04d}' for i in range(1, n_samples + 1)],
        'Description': np.random.choice(all_products, n_samples),
        'UnitPrice': np.random.uniform(5.0, 500.0, n_samples).round(2),
        'Country': np.random.choice(countries, n_samples)
    }
    
    return pd.DataFrame(data)

# Display basic information about the dataset
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

In [ ]:
# Explore the dataset
print("Dataset Exploration")
print("=" * 50)

# Basic statistics
print(f"Total records: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Check for missing values
print("\nMissing Values:")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nDuplicate records: {duplicates:,}")

# Data types
print("\nData Types:")
print(df.dtypes)

# Unique values in categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
print("\nUnique values in categorical columns:")
for col in categorical_columns:
    print(f"{col}: {df[col].nunique():,} unique values")

In [ ]:
# Data cleaning and preprocessing
print("Data Cleaning Process")
print("=" * 50)

# Create a copy for cleaning
df_clean = df.copy()
initial_rows = len(df_clean)

# 1. Remove duplicates
df_clean = df_clean.drop_duplicates()
print(f"Removed {initial_rows - len(df_clean):,} duplicate rows")

# 2. Handle missing values
# Remove rows with missing critical information
critical_columns = ['StockCode', 'Description']
for col in critical_columns:
    if col in df_clean.columns:
        before = len(df_clean)
        df_clean = df_clean.dropna(subset=[col])
        removed = before - len(df_clean)
        if removed > 0:
            print(f"Removed {removed:,} rows with missing {col}")

# Fill missing values for non-critical columns
if 'Country' in df_clean.columns:
    df_clean['Country'] = df_clean['Country'].fillna('Unknown')

if 'UnitPrice' in df_clean.columns:
    # Remove rows with invalid prices
    before = len(df_clean)
    df_clean = df_clean[df_clean['UnitPrice'] > 0]
    removed = before - len(df_clean)
    if removed > 0:
        print(f"Removed {removed:,} rows with invalid prices")

# 3. Standardize text data
if 'Description' in df_clean.columns:
    # Clean description text
    df_clean['Description'] = df_clean['Description'].astype(str)
    df_clean['Description'] = df_clean['Description'].str.strip()
    df_clean['Description'] = df_clean['Description'].str.title()
    
    # Remove very short descriptions
    before = len(df_clean)
    df_clean = df_clean[df_clean['Description'].str.len() >= 3]
    removed = before - len(df_clean)
    if removed > 0:
        print(f"Removed {removed:,} rows with very short descriptions")

# 4. Standardize stock codes
if 'StockCode' in df_clean.columns:
    df_clean['StockCode'] = df_clean['StockCode'].astype(str).str.strip().str.upper()

# 5. Standardize country names
if 'Country' in df_clean.columns:
    df_clean['Country'] = df_clean['Country'].str.strip().str.title()
    
    # Standardize common country name variations
    country_mapping = {
        'Usa': 'USA',
        'United States': 'USA',
        'Us': 'USA',
        'Uk': 'UK',
        'United Kingdom': 'UK',
        'Britain': 'UK'
    }
    df_clean['Country'] = df_clean['Country'].replace(country_mapping)

print(f"\nFinal dataset shape: {df_clean.shape}")
print(f"Removed {initial_rows - len(df_clean):,} rows in total ({((initial_rows - len(df_clean)) / initial_rows * 100):.1f}%)")

In [ ]:
# Data quality assessment
print("Data Quality Assessment")
print("=" * 50)

# Check for remaining issues
print("Remaining missing values:")
print(df_clean.isnull().sum())

print("\nData type consistency:")
print(df_clean.dtypes)

# Price distribution
if 'UnitPrice' in df_clean.columns:
    print("\nPrice statistics:")
    print(df_clean['UnitPrice'].describe())
    
    # Check for outliers
    Q1 = df_clean['UnitPrice'].quantile(0.25)
    Q3 = df_clean['UnitPrice'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df_clean[(df_clean['UnitPrice'] < lower_bound) | (df_clean['UnitPrice'] > upper_bound)]
    print(f"Price outliers: {len(outliers):,} ({len(outliers)/len(df_clean)*100:.1f}%)")

# Top countries
if 'Country' in df_clean.columns:
    print("\nTop 10 countries:")
    print(df_clean['Country'].value_counts().head(10))

# Description length distribution
if 'Description' in df_clean.columns:
    df_clean['description_length'] = df_clean['Description'].str.len()
    print("\nDescription length statistics:")
    print(df_clean['description_length'].describe())
    
    # Most common words in descriptions
    all_descriptions = ' '.join(df_clean['Description'].astype(str))
    words = re.findall(r'\b\w+\b', all_descriptions.lower())
    word_freq = pd.Series(words).value_counts()
    print("\nTop 10 most common words in descriptions:")
    print(word_freq.head(10))

In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Price distribution
if 'UnitPrice' in df_clean.columns:
    axes[0, 0].hist(df_clean['UnitPrice'], bins=50, alpha=0.7, color='skyblue')
    axes[0, 0].set_title('Price Distribution')
    axes[0, 0].set_xlabel('Unit Price')
    axes[0, 0].set_ylabel('Frequency')

# Country distribution
if 'Country' in df_clean.columns:
    top_countries = df_clean['Country'].value_counts().head(10)
    axes[0, 1].bar(range(len(top_countries)), top_countries.values, color='lightcoral')
    axes[0, 1].set_title('Top 10 Countries')
    axes[0, 1].set_xlabel('Country')
    axes[0, 1].set_ylabel('Number of Products')
    axes[0, 1].set_xticks(range(len(top_countries)))
    axes[0, 1].set_xticklabels(top_countries.index, rotation=45)

# Description length distribution
if 'description_length' in df_clean.columns:
    axes[1, 0].hist(df_clean['description_length'], bins=30, alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('Description Length Distribution')
    axes[1, 0].set_xlabel('Description Length (characters)')
    axes[1, 0].set_ylabel('Frequency')

# Price by country (top 5 countries)
if 'UnitPrice' in df_clean.columns and 'Country' in df_clean.columns:
    top_5_countries = df_clean['Country'].value_counts().head(5).index
    price_by_country = [df_clean[df_clean['Country'] == country]['UnitPrice'].values for country in top_5_countries]
    axes[1, 1].boxplot(price_by_country, labels=top_5_countries)
    axes[1, 1].set_title('Price Distribution by Top 5 Countries')
    axes[1, 1].set_xlabel('Country')
    axes[1, 1].set_ylabel('Unit Price')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Save the plot
plt.savefig('../static/images/data_exploration.png', dpi=300, bbox_inches='tight')
print("Visualization saved to ../static/images/data_exploration.png")

In [ ]:
# Save cleaned data to database
print("Saving cleaned data to database...")

try:
    # Initialize database service
    db_service = DatabaseService()
    
    # Save cleaned data to CSV first
    cleaned_csv_path = '../data/cleaned_products.csv'
    df_clean.to_csv(cleaned_csv_path, index=False)
    print(f"Cleaned data saved to {cleaned_csv_path}")
    
    # Load data into database
    products_loaded = db_service.load_products_from_csv(cleaned_csv_path)
    print(f"Successfully loaded {products_loaded:,} products into database")
    
    # Verify data in database
    sample_products = db_service.get_products(limit=5)
    print("\nSample products from database:")
    for product in sample_products:
        print(f"  {product['stock_code']}: {product['description']} - ${product['unit_price']:.2f}")
    
except Exception as e:
    print(f"Error saving to database: {e}")
    print("Data cleaning completed but not saved to database.")

print("\nData cleaning process completed successfully!")
print(f"Final dataset: {len(df_clean):,} products ready for vectorization")

In [ ]:
# Generate summary report
summary_report = f"""
E-COMMERCE DATASET CLEANING SUMMARY REPORT
==========================================

Dataset Information:
- Original records: {initial_rows:,}
- Final records: {len(df_clean):,}
- Records removed: {initial_rows - len(df_clean):,} ({((initial_rows - len(df_clean)) / initial_rows * 100):.1f}%)
- Columns: {len(df_clean.columns)}

Data Quality:
- Missing values: {df_clean.isnull().sum().sum()}
- Duplicate records: 0 (removed)
- Unique products: {df_clean['StockCode'].nunique() if 'StockCode' in df_clean.columns else 'N/A'}
- Countries: {df_clean['Country'].nunique() if 'Country' in df_clean.columns else 'N/A'}

Price Statistics:
- Min price: ${df_clean['UnitPrice'].min():.2f if 'UnitPrice' in df_clean.columns else 'N/A'}
- Max price: ${df_clean['UnitPrice'].max():.2f if 'UnitPrice' in df_clean.columns else 'N/A'}
- Average price: ${df_clean['UnitPrice'].mean():.2f if 'UnitPrice' in df_clean.columns else 'N/A'}
- Median price: ${df_clean['UnitPrice'].median():.2f if 'UnitPrice' in df_clean.columns else 'N/A'}

Next Steps:
1. Create product vectors for similarity search
2. Set up vector database (Pinecone)
3. Train recommendation model
4. Implement API endpoints

Files Generated:
- ../data/cleaned_products.csv
- ../static/images/data_exploration.png
- Database populated with clean product data
"""

print(summary_report)

# Save report to file
with open('../data/cleaning_report.txt', 'w') as f:
    f.write(summary_report)

print("\nSummary report saved to ../data/cleaning_report.txt")